In [107]:
%matplotlib inline 

import math
import copy
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch import optim, nn

import model
import model_utils
import preprocessing_utils

import geopandas as gpd
import pandas as pd
from shapely.geometry import LineString
import contextily as ctx
import matplotlib.pyplot as plt

from os.path import dirname, abspath
import os
import sys

#to be put under model.py when I refactor
from tqdm.notebook import tqdm

ModuleNotFoundError: No module named 'geopandas'

In [72]:
torch.manual_seed(1234)

In [73]:
device = torch.device('cpu') # Heroku only have cpu

num_timesteps_input = 7 # Default is 30 minutes
num_timesteps_output = 4 # Default is 10 minutes

loss_criterion = nn.MSELoss()

In [74]:
os.getcwd()

'C:\\Users\\daryl\\Desktop\\SUTD school materials\\T7-TheoryAndPracticeOfDeepLearning\\Big Project\\traffic_prediction\\interactive-app'

In [75]:
raw_dir = os.path.join(os.getcwd(), 'data', 'raw')
process_dir = os.path.join(os.getcwd(), 'data', 'processed')

# overwrite = False means that the processing function will only run if the process data files do not exist
# overwrite = True => functions will run regardless
preprocessing_utils.processed(raw_dir, process_dir, overwrite=True)
A, X, metadata, cat2index, means, stds = preprocessing_utils.load(process_dir)


test_original_data = X


test_input, test_target = preprocessing_utils.generate_dataset(test_original_data,
                                           num_timesteps_input=num_timesteps_input,
                                           num_timesteps_output=num_timesteps_output)

# input shape (num_samples ,num_vertices, num_timesteps_window, num_features)
# output shape (num_samples ,num_vertices, num_timesteps_window)
adj_mat = preprocessing_utils.get_normalized_adj(A)
adj_mat = torch.from_numpy(adj_mat).to(device)

Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_00_10.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_05_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_10_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_15_09.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_prediction\interactive-app\data\raw\Thu_Apr_1_2021\15_20_10.json
Processing C:\Users\daryl\Desktop\SUTD school materials\T7-TheoryAndPracticeOfDeepLearning\Big Project\traffic_predictio

In [76]:
X.shape

(53, 3, 7)

In [77]:
# indices = [(i, i + (num_timesteps_input + num_timesteps_output)) for i
#                in range(X.shape[2] - (
#                 num_timesteps_input + num_timesteps_output) + 1)]
i = 0
indices = [(i, i + (num_timesteps_input + num_timesteps_output))]

# Save samples
features, target = [], []
for i, j in indices:
    features.append(
        X[:, :, i: i + num_timesteps_input].transpose(
            (0, 2, 1)))
    target.append(X[:, 0, i + num_timesteps_input: j])

In [78]:
test_input = torch.from_numpy(np.array(features))

In [79]:
test_target = torch.from_numpy(np.array(target))

In [80]:
# Load model
saved_models_path = os.path.join(dirname(os.getcwd()), 'saved_models', 'last_saved_model.txt')
with open(saved_models_path) as f:
    saved_model = f.read()

latest_model_path = os.path.join(dirname(os.getcwd()), saved_model)
checkpoint = torch.load(latest_model_path, map_location=None)
model_stgcn = model.Stgcn_Model(checkpoint['model_nodes_num'], checkpoint['model_features_num'], checkpoint['model_input_timesteps'], checkpoint['model_num_output'])
model_stgcn.load_state_dict(checkpoint['state_dict'])
optimizer = optim.Adam(model_stgcn.parameters(), lr=checkpoint['model_lr'])
optimizer = optimizer.load_state_dict(checkpoint['opti_state_dict'])

In [81]:
loaded_model = model_stgcn
loaded_optimizer = optimizer

In [82]:
test_input.shape

torch.Size([1, 53, 7, 3])

In [83]:
test_target.shape

torch.Size([1, 53, 0])

In [95]:
torch.manual_seed(1234)


predicted = model_utils.predict(loaded_model, test_input, adj_mat)
predicted_denorm = preprocessing_utils.denormalize(predicted, stds[0], means[0])

In [96]:
predicted

tensor([[[-1.0907, -1.0230, -0.9304, -0.8908],
         [ 0.3436,  0.2000,  0.1298,  0.0950],
         [-0.6031, -0.4171, -0.2409, -0.0788],
         [ 0.3337,  0.2361,  0.1865,  0.1483],
         [ 0.9121,  0.9326,  0.9228,  0.9147],
         [-0.4083, -0.5070, -0.5739, -0.5819],
         [ 0.8408,  0.7398,  0.6884,  0.6627],
         [-0.7209, -0.6660, -0.6274, -0.5984],
         [ 0.7964,  0.8453,  0.8413,  0.8343],
         [-0.6207, -0.6285, -0.6224, -0.6137],
         [ 0.3557,  0.3293,  0.3090,  0.3042],
         [-0.8883, -0.9533, -0.9888, -0.9856],
         [-0.3670, -0.1425,  0.0149,  0.0708],
         [ 0.7834,  0.6583,  0.5853,  0.5092],
         [-0.1691, -0.2074, -0.2067, -0.2188],
         [ 0.3602,  0.3151,  0.2924,  0.2676],
         [-0.2834, -0.3301, -0.3577, -0.3538],
         [-1.8811, -1.7838, -1.6867, -1.6231],
         [-0.5618, -0.3278, -0.1591, -0.0832],
         [ 0.2119,  0.3058,  0.3308,  0.3574],
         [-0.2687, -0.3045, -0.3125, -0.2845],
         [ 0.

In [97]:
predicted_denorm

tensor([[[2.7098, 2.7586, 2.8253, 2.8539],
         [3.7436, 3.6401, 3.5895, 3.5644],
         [3.0612, 3.1953, 3.3223, 3.4391],
         [3.7365, 3.6661, 3.6304, 3.6029],
         [4.1534, 4.1682, 4.1611, 4.1553],
         [3.2017, 3.1305, 3.0823, 3.0765],
         [4.1020, 4.0292, 3.9921, 3.9736],
         [2.9764, 3.0159, 3.0438, 3.0646],
         [4.0700, 4.1052, 4.1023, 4.0973],
         [3.0485, 3.0430, 3.0474, 3.0536],
         [3.7523, 3.7333, 3.7187, 3.7152],
         [2.8557, 2.8088, 2.7832, 2.7856],
         [3.2315, 3.3933, 3.5067, 3.5470],
         [4.0606, 3.9705, 3.9178, 3.8630],
         [3.3741, 3.3465, 3.3470, 3.3382],
         [3.7556, 3.7231, 3.7067, 3.6888],
         [3.2917, 3.2580, 3.2382, 3.2410],
         [2.1401, 2.2103, 2.2802, 2.3261],
         [3.0910, 3.2597, 3.3813, 3.4360],
         [3.6487, 3.7163, 3.7344, 3.7536],
         [3.3023, 3.2764, 3.2707, 3.2909],
         [3.5489, 3.4986, 3.4659, 3.4647],
         [3.0143, 3.0513, 3.0645, 3.0843],
         [3

In [100]:
predicted_denorm = predicted_denorm.numpy()

In [102]:
predicted_denorm

array([[[2.7098315, 2.758624 , 2.825325 , 2.8538733],
        [3.743645 , 3.6401002, 3.589492 , 3.5644426],
        [3.0612454, 3.1953342, 3.3223026, 3.439126 ],
        [3.7365055, 3.666101 , 3.630376 , 3.6028795],
        [4.153403 , 4.168185 , 4.161093 , 4.1552706],
        [3.201654 , 3.1305366, 3.0822837, 3.0765405],
        [4.102011 , 4.029174 , 3.9921384, 3.973587 ],
        [2.9763768, 3.0158904, 3.043758 , 3.0646274],
        [4.0699887, 4.1052203, 4.102341 , 4.097325 ],
        [3.0485349, 3.0429738, 3.0473654, 3.053603 ],
        [3.752307 , 3.7333229, 3.7186573, 3.7152262],
        [2.8557115, 2.8088136, 2.783217 , 2.7855885],
        [3.2314637, 3.3932502, 3.5067198, 3.5469804],
        [4.0606165, 3.9704814, 3.9178426, 3.8629813],
        [3.3740702, 3.346495 , 3.3470073, 3.3382444],
        [3.7555683, 3.7231069, 3.7066944, 3.6888347],
        [3.2917151, 3.2579975, 3.2381659, 3.240961 ],
        [2.1401005, 2.2102618, 2.2802258, 2.3260736],
        [3.0910234, 3.259688

In [103]:
predicted_denorm.shape

(1, 53, 4)

In [105]:
timestep_speedbands = predicted_denorm.reshape(predicted_denorm.shape[2], predicted_denorm.shape[1])

In [106]:
timestep_speedbands

array([[2.7098315, 2.758624 , 2.825325 , 2.8538733, 3.743645 , 3.6401002,
        3.589492 , 3.5644426, 3.0612454, 3.1953342, 3.3223026, 3.439126 ,
        3.7365055, 3.666101 , 3.630376 , 3.6028795, 4.153403 , 4.168185 ,
        4.161093 , 4.1552706, 3.201654 , 3.1305366, 3.0822837, 3.0765405,
        4.102011 , 4.029174 , 3.9921384, 3.973587 , 2.9763768, 3.0158904,
        3.043758 , 3.0646274, 4.0699887, 4.1052203, 4.102341 , 4.097325 ,
        3.0485349, 3.0429738, 3.0473654, 3.053603 , 3.752307 , 3.7333229,
        3.7186573, 3.7152262, 2.8557115, 2.8088136, 2.783217 , 2.7855885,
        3.2314637, 3.3932502, 3.5067198, 3.5469804, 4.0606165],
       [3.9704814, 3.9178426, 3.8629813, 3.3740702, 3.346495 , 3.3470073,
        3.3382444, 3.7555683, 3.7231069, 3.7066944, 3.6888347, 3.2917151,
        3.2579975, 3.2381659, 3.240961 , 2.1401005, 2.2102618, 2.2802258,
        2.3260736, 3.0910234, 3.2596884, 3.3813057, 3.4360013, 3.648703 ,
        3.7163389, 3.734406 , 3.7535875, 3.30231

In [ ]:


def plotGeoPerformance(metadata, speedbands):
    df = pd.DataFrame(metadata).transpose()
    df["speedbands"] = speedbands
    loc = df["start_pos"] + " " + df["end_pos"]
    linestrings = loc.apply(loc_to_linestring)
    gdf = gpd.GeoDataFrame(df, geometry=linestrings, crs="EPSG:4326")
    gdf = gdf.to_crs('EPSG:3857')
    fig, ax = plt.subplots(figsize=(10, 10))
    gdf.plot(ax=ax, column="speedbands", legend=True, cmap="OrRd", legend_kwds={'label': 'speedbands'})
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ctx.add_basemap(ax)